In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from quick_pp.objects import Project

# Load well from saved file
project = "MOCK_carbonate"
project_path = rf"data\04_project\{project}.qppp"
project = Project().load(project_path)
project.get_well_names()

In [40]:
# Load data
well_name = 'M3'
well = project.get_well(well_name)
well_data = well.data.copy()
well_data.dropna(subset=['NPHI', 'RHOB', 'RT', 'GR'], inplace=True)

# Quick PP Interpretation

In [41]:
from quick_pp.lithology.carbonate import Carbonate
from quick_pp.porosity import neu_den_xplot_poro, density_porosity, rho_matrix
from quick_pp.qaqc import badhole_flagging, mask_outside_threshold, neu_den_xplot_hc_correction
from quick_pp.saturation import *
from quick_pp.plotter import *
from quick_pp.permeability import *
from quick_pp.ressum import *
from quick_pp.rock_type import rock_typing, estimate_vsh_gr
from quick_pp.plotter import plotly_log
from quick_pp.utils import zone_flagging, min_max_line

## Estimate Litholoy

In [43]:
carbonate_type =  'limestone'  # 'dolostone'  #
model = 'single'  # 'double'  #
method = 'neu_den'  # 'pef_den'  #

# Clean up data
well_data = badhole_flagging(well_data)
# well_data = mask_outside_threshold(well_data, fill=True)

# Initialize lithology model
args = {
    'litho_model': 'carb',
    'dry_sand_point': (.0, 2.71) if carbonate_type == 'limestone' else (.0, 2.81),  # Dry calcite point
    'dry_clay_point': (.3, 2.7),
    'silt_line_angle': 116,
    'wet_clay_point': (0.43, 2.6),
    'sw_water_salinity': 5000,
    'sw_m': 1.8,
    'sw_n': 2,
    'hc_corr_angle': 50,
    'hc_buffer': 0.1,
    'ressum_cutoffs': dict(
        VSHALE=.5,
        PHIT=0,
        SWT=1
    ),
}
vsh_gr = estimate_vsh_gr(well_data['GR'], min_gr=0, max_gr=50)
carb_model = Carbonate(**args)
vclw, vcalc, vdolo = carb_model.estimate_lithology(
    well_data['NPHI'], well_data['RHOB'], gr=well_data['GR'], vsh_gr=vsh_gr,  # pef=well_data['PEF'],
    model=model, method=method, normalize= True, carbonate_type=carbonate_type
)
args.update(carb_model.__dict__)
well.update_config(args)  # Save lithology model to well

# Implement hydrocarbon correction
nphihc, rhobhc, hc_flag = neu_den_xplot_hc_correction(
    well_data['NPHI'], well_data['RHOB'], gr=well_data['GR'], vsh_gr=vsh_gr,
    dry_sand_point=args['dry_sand_point'],
    dry_clay_point=args['dry_clay_point'],
    corr_angle=args['hc_corr_angle'], buffer=args['hc_buffer']
)

# Estimate lithology
carb_model = Carbonate(**args)
vclw, vcalc, vdolo = carb_model.estimate_lithology(
    nphihc, rhobhc,  gr=well_data['GR'], vsh_gr=vsh_gr,  # pef=well_data['PEF'], 
    model=model, method=method, normalize= True, carbonate_type=carbonate_type
)

# Estimate rock types
rock_flag = rock_typing(vclw, higher_is_better=False)

# Update ZONES
well_data = zone_flagging(well_data)
zones = well_data.ZONES

In [ ]:
copy_args = args.copy()
copy_args.pop('silt_line_angle')
copy_args['dry_silt_point'] = (0.2, 2.65)
print(copy_args)
neutron_density_xplot(well_data['NPHI'], well_data['RHOB'], **copy_args)

In [ ]:
neutron_density_xplot(nphihc, rhobhc, **copy_args)

In [ ]:
from quick_pp.rock_type import estimate_vsh_gr

vsh_gr = estimate_vsh_gr(well_data['GR'], alpha=0.1)
vsh_gr2 = estimate_vsh_gr(well_data['GR'], min_gr=0, max_gr=50)

fig, axs = plt.subplots(2, 1, figsize=(20, 3), sharex=True)

axs[0].plot(well_data.DEPTH, vsh_gr, label='vsh_gr')
axs[0].plot(well_data.DEPTH, vsh_gr2, label='vsh_gr2')
axs[0].set_ylim(0, 1)
axs[0].legend()

axs[1].plot(well_data.DEPTH, well_data['GR'], label='GR')
axs[1].legend()

In [48]:
# Specify depth of interest
min_depth = well_data.DEPTH.min()
max_depth = well_data.DEPTH.max()

## Estimate Porosity

In [ ]:
# PHID needs unnormalized lithology
vclw_un, vcalc_un, vdolo_un = Carbonate(**args).estimate_lithology(
    nphihc, rhobhc, gr=well_data['GR'], pef=well_data['PEF'],
    model=model, method=method, normalize= False, carbonate_type=carbonate_type
)
rho_ma = rho_matrix(vclay=vclw_un, vcalc=vcalc_un, vdolo=vdolo_un)
phid = phit = density_porosity(rhobhc, rho_ma)

plt.figure(figsize=(20, 2))
plt.plot(phid, label='PHID')

## Estimate Permeability

In [ ]:
import matplotlib.ticker as ticker

# Estimate permeability
perm = choo_permeability(vclw, 0, phit,  B=6, A=5e6)
Swirr = np.where(
    rock_flag == 1, 0.005, np.where(
        rock_flag == 2, 0.2, np.where(
            rock_flag == 3, 0.3, 1
        )
    )
)
perm_kc = kozeny_carman_permeability(phit, S=.5)
perm_t = timur_permeability(phit, Swirr)
perm_c = coates_permeability(phit, Swirr, a=.5)
perm_tx = tixier_permeability(phit, Swirr)

# Plotting
fig, ax = plt.subplots(figsize=(20, 3))
ax.plot(well_data.DEPTH, perm, label='Choo')
ax.plot(well_data.DEPTH, perm_kc, label='KC')
ax.plot(well_data.DEPTH, perm_t, label='Timur')
ax.plot(well_data.DEPTH, perm_c, label='Coates')
ax.plot(well_data.DEPTH, perm_tx, label='Tixier')
ax.set_frame_on(False)
ax.set_yscale('log')
ax.set_ylim(1e-2, 1e5)
ax.yaxis.set_major_formatter(ticker.FuncFormatter(
    lambda x, pos: ('{{:.{:1d}f}}'.format(int(np.maximum(-np.log10(x), 0)))).format(x)))
ax.set_xlim(min_depth, max_depth)
ax.legend(bbox_to_anchor=(1.04, 1), loc="upper left")

## Estimate Water Saturation

In [ ]:
# Debug water saturation
water_salinity = 5000
m = args['sw_m']

temp_grad = estimate_temperature_gradient(well_data['DEPTH'], 'imperial')
rw = estimate_rw_temperature_salinity(temp_grad, water_salinity)
qv = estimate_qv(vclw, phit, cec_clay=.1)

swt = waxman_smits_saturation(well_data['RT'], rw, phit, qv)

# Plotting
fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, sharex=True, figsize=(20, 4))

ax1.plot(well_data.DEPTH, qv, label='Qv')
ax1.set_ylim(-1, 10)
ax1.legend()

ax2.plot(well_data.DEPTH, well_data['RT'], label='RT')
ax2.plot(well_data.DEPTH, rw, label='rw_salinity')
ax2.plot(well_data.DEPTH, rw, label='rw')
ax2.set_yscale('log')
ax2.legend()

ax3.plot(well_data.DEPTH, swt, label='SWT')
ax3.legend()

fig.tight_layout()

## Update Data

In [52]:
# Update data in the project
well_data['NPHI_HC'] = nphihc
well_data['RHOB_HC'] = rhobhc
well_data['HC_FLAG'] = hc_flag
well_data['VCALC'] = vcalc
well_data['VDOLO'] = vdolo
well_data['VCLW'] = vclw
well_data['PHIT'] = phit
well_data['PHID'] = phid
well_data['RW'] = rw
well_data['Qv'] = qv
well_data['M'] = m
well_data['SWT'] = swt
well_data['PERM'] = perm
well_data['ROCK_FLAG'] = rock_flag
well_data['VHC'] = phit * (1 - swt)

In [53]:
# Estimate reservoir summary
ressum_df = calc_reservoir_summary(well_data.DEPTH, vclw, phit, swt, perm, zones, cutoffs=args['ressum_cutoffs'])

In [ ]:
# Plot the results
well_data.drop(columns=['VOIL', 'VGAS'], errors='ignore', inplace=True)
fig = plotly_log(well_data, depth_uom=well.depth_uom)
fig.show(config=dict(scrollZoom=True))
# fig.write_html(rf"{well_name}_log.html", config=dict(scrollZoom=True))

## Multimineral

In [ ]:
from quick_pp.lithology.multi_mineral import MultiMineral

vol_quartz, vol_calcite, vol_dolomite, vol_shale, vol_mud = MultiMineral().estimate_lithology(
    well_data['GR'], well_data['NPHI_HC'], well_data['RHOB_HC'], well_data['PEF'], well_data['DTC'])

multimin = pd.DataFrame(
    {'VSAND': vol_quartz, 'VCLW': vol_shale, 'VDOLO': vol_dolomite, 'VCALC': vol_calcite},
    index=well_data.DEPTH
)
multimin = well_data.merge(multimin, how='left', on='DEPTH', suffixes=('_ORI', ''))

In [ ]:
fig, axs = plt.subplots(nrows=3, sharex=True, figsize=(20, 7))
axs[0].plot(multimin.DEPTH, multimin['GR'], label='GR')
axs[0].legend()
axs[1].plot(multimin.DEPTH, multimin['DTC'], label='DTC')
axs[1].legend()
axs[2].plot(multimin.DEPTH, multimin['PEF'], label='PEF')
axs[2].legend()


In [ ]:
# Plot the results
multimin.drop(columns=['VOIL', 'VGAS'], errors='ignore', inplace=True)
fig = plotly_log(multimin, depth_uom=well.depth_uom)
fig.show(config=dict(scrollZoom=True))
# fig.write_html(rf"{well_name}_log.html", config=dict(scrollZoom=True))

In [ ]:
from quick_pp.qaqc import quick_qc

test, summary_df, dist_fig, depth_fig = quick_qc(well_data)

dist_fig.show()
depth_fig.show()

In [59]:
# # Save the results to the well
# well.update_data(well_data)
# well.update_ressum(ressum_df)
# project.save_well(well)

In [60]:
# project.save()